In [1]:
import nltk
from nltk.tokenize import word_tokenize
from src.utils.stopwords_manager import StopwordsManager
from src.utils.croatian_stopwords import croatian_stopwords
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from numpy import e as np_e
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



In [2]:
%%sql
SELECT * FROM reviews_and_ratings_translated_no_emojis;

,translated_review,rating
0,Odličan restoran s klasičnim hrvatskim dodirom...,5
1,"Gostoljubiva, ljubazna, dobra hrana, velika vr...",5
2,Razumna cijena.!Lijep ambijent naručio kozice ...,5
3,Okus u redu.Vrlo čist restoran,4
4,"Bilo je to savršeno dobro ugodno vjenčanje, pu...",5
...,...,...
202,NAJBOLJI RESTORAN U ZAGREBU,5
203,Preporuka za otići na gablec,5
204,Dobar restoran po povoljnim cijenama.,4
205,Parkirno mjesto puno mogućnosti parkiranja na ...,5


In [3]:

nltk.download('punkt')

# Initialize the stopwords manager with Croatian stopwords
stopwords_manager = StopwordsManager(base_stopwords=croatian_stopwords)

# Function to clean text using StopwordsManager
def clean_text(text):
    word_tokens = word_tokenize(text)
    return stopwords_manager.get_clean_text_for_sentiment(' '.join(word_tokens))

# Apply the cleaning function to the 'text_column' of df_sql1
df_sql1['cleaned_text'] = df_sql1['translated_review'].apply(clean_text)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ivan.zeljeznjak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
df_sql1

,translated_review,rating,cleaned_text
0,Odličan restoran s klasičnim hrvatskim dodirom...,5,odličan klasičnim hrvatskim dodirom.lako prona...
1,"Gostoljubiva, ljubazna, dobra hrana, velika vr...",5,"gostoljubiva , ljubazna , dobra , velika vrije..."
2,Razumna cijena.!Lijep ambijent naručio kozice ...,5,razumna cijena . ! lijep ambijent naručio kozi...
3,Okus u redu.Vrlo čist restoran,4,okus redu.vrlo čist
4,"Bilo je to savršeno dobro ugodno vjenčanje, pu...",5,"to savršeno dobro ugodno vjenčanje , prostora ..."
...,...,...,...
202,NAJBOLJI RESTORAN U ZAGREBU,5,najbolji zagrebu
203,Preporuka za otići na gablec,5,preporuka otići gablec
204,Dobar restoran po povoljnim cijenama.,4,dobar povoljnim cijenama .
205,Parkirno mjesto puno mogućnosti parkiranja na ...,5,parkirno mjesto mogućnosti parkiranja parkiran...


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df_sql1['cleaned_text'], df_sql1['rating'], test_size=1/np_e)

In [6]:
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Train the model with the training data
model.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [7]:

# Predict the labels for the test set
y_pred = model.predict(X_test)

# Print out the classification report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           1       0.00      0.00      0.00         7
           2       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00        17
           5       0.60      1.00      0.75        46

    accuracy                           0.60        77
   macro avg       0.12      0.20      0.15        77
weighted avg       0.36      0.60      0.45        77


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
#placeholder